In [ ]:
import pandas as pd
import numpy as np

np.random.seed(1212)

import keras #Keras is a high-level, deep learning API developed by Google for implementing neural networks.
from keras.models import Model #a model is a function with learnable parameters that maps an input to an output.
from keras.layers import * #define the architecture and functionality of neural network models
from keras import optimizers  #change the attributes of machine/deep learning model such as weights and learning rate in order to reduce the losses.

In [ ]:
from google.colab import files


uploaded = files.upload()

Saving test.csv to test.csv


In [ ]:
from google.colab import files


uploaded = files.upload()

Saving train.csv to train.csv


In [ ]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [ ]:
df_train.head() # 784 features, 1 label

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_features = df_train.iloc[:, 1:785] #contain the training data without the target (label) column.
df_label = df_train.iloc[:, 0] #contain the target (label) column

X_test = df_test.iloc[:, 0:784] # selects all rows and column

print(X_test.shape) #get number of rows and columns

(28000, 784)


In [ ]:
from sklearn.model_selection import train_test_split                     #split data into training and cross-validation (cv) sets
X_train, X_cv, y_train, y_cv = train_test_split(df_features, df_label,
                                                test_size = 0.2,         #20% data will used for cross validation
                                                random_state = 1212)     # ensures reproducibility of the results.

#x_train - 80% df_features, x_cv - 20% df_feature,
X_train = X_train.values.reshape(33600, 784) #(33600, 784)
X_cv = X_cv.values.reshape(8400, 784) #(8400, 784)

X_test = X_test.values.reshape(28000, 784)  #2-dimensional input data

In [ ]:
print((min(X_train[1]), max(X_train[1])))
#original pixel values range from 0 to 255 ( 8-bit grayscale images)

In [ ]:
# Feature Normalization
X_train = X_train.astype('float32'); X_cv= X_cv.astype('float32'); X_test = X_test.astype('float32')
X_train /= 255; X_cv /= 255; X_test /= 255   #dividing by 255 scales the values to a range between 0 and 1.

# Convert labels to One Hot Encoded
num_digits = 10
y_train = keras.utils.to_categorical(y_train, num_digits)  # convert categorical labels into a binary representation
y_cv = keras.utils.to_categorical(y_cv, num_digits)

In [ ]:
# Printing 2 examples of labels after conversion
print(y_train[0]) # 2
print(y_train[3]) # 7

[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


In [ ]:
# Input Parameters
n_input = 784 # number of features in the input data - match the number of columns in X_train, X_cv, x_test
n_hidden_1 = 300  #300 neurons in the first hidden layer
n_hidden_2 = 100
n_hidden_3 = 100
n_hidden_4 = 200
num_digits = 10   #categories in the classification task  (0 to 9)

In [ ]:
Inp = Input(shape=(784,))   #the batch size will be determined during the model's training.
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x) #'softmax' activation function is used in the output layer for multi-class classification to produce class probability, num_digits = 10(digits 0 -9)


In [ ]:
# Our model would have '6' layers - input layer, 4 hidden layer and 1 output layer
model = Model(Inp, output)
model.summary() # We have 297,910 parameters to estimate. Parameters in a neural network are the weights and biases of the neurons in each layer.

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 784)]             0         
                                                                 
 Hidden_Layer_1 (Dense)      (None, 300)               235500    
                                                                 
 Hidden_Layer_2 (Dense)      (None, 100)               30100     
                                                                 
 Hidden_Layer_3 (Dense)      (None, 100)               10100     
                                                                 
 Hidden_Layer_4 (Dense)      (None, 200)               20200     
                                                                 
 Output_Layer (Dense)        (None, 10)                2010      
                                                                 
Total params: 297,910
Trainable params: 297,910
Non-trainable

In [ ]:
# Insert Hyperparameters
learning_rate = 0.1
training_epochs = 20 # Each epoch consists of one forward pass (prediction) and one backward pass (weight update) for all the training samples.
batch_size = 100     # Number of samples that are processed together in parallel
sgd = optimizers.SGD(lr=learning_rate)  #SGD - Stochastic Gradient Descent, updates the model's weights based on the gradients of the loss function with respect to the weights.

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
# We rely on the plain vanilla Stochastic Gradient Descent as our optimizing methodology
model.compile(loss='categorical_crossentropy', #Categorical Crossentropy is a common loss function used for multi-class classification problem
              optimizer='sgd',
              metrics=['accuracy'])  #accuracy metric measures the percentage of correctly predicted samples out of the total samples during training

In [ ]:
history1 = model.fit(X_train, y_train,
                     batch_size = batch_size,
                     epochs = training_epochs,
                     verbose = 2,
                     validation_data=(X_cv, y_cv))

#X_train represents the training features (input data), and y_train represents the corresponding one-hot encoded labels (target values).
#batch size is number of samples that will be processed in each update of the model's weights during a single training iteration.
#The training_epochs hyperparameter determines the number of times the entire training dataset (X_train and y_train) will be passed through the neural network during the training process.
#A value of 2 means that progress will be displayed for each epoch, showing the training loss and metrics.
# the model will be evaluated on this data after each epoch to monitor its performance on unseen data and detect overfitting.

Epoch 1/20
336/336 - 4s - loss: 1.8235 - accuracy: 0.4861 - val_loss: 0.9412 - val_accuracy: 0.7963 - 4s/epoch - 11ms/step
Epoch 2/20
336/336 - 3s - loss: 0.5939 - accuracy: 0.8494 - val_loss: 0.4369 - val_accuracy: 0.8793 - 3s/epoch - 8ms/step
Epoch 3/20
336/336 - 2s - loss: 0.3876 - accuracy: 0.8887 - val_loss: 0.3506 - val_accuracy: 0.9011 - 2s/epoch - 7ms/step
Epoch 4/20
336/336 - 5s - loss: 0.3260 - accuracy: 0.9044 - val_loss: 0.3119 - val_accuracy: 0.9118 - 5s/epoch - 14ms/step
Epoch 5/20
336/336 - 3s - loss: 0.2908 - accuracy: 0.9141 - val_loss: 0.2867 - val_accuracy: 0.9152 - 3s/epoch - 8ms/step
Epoch 6/20
336/336 - 2s - loss: 0.2636 - accuracy: 0.9233 - val_loss: 0.2590 - val_accuracy: 0.9258 - 2s/epoch - 7ms/step
Epoch 7/20
336/336 - 2s - loss: 0.2438 - accuracy: 0.9288 - val_loss: 0.2443 - val_accuracy: 0.9280 - 2s/epoch - 7ms/step
Epoch 8/20
336/336 - 3s - loss: 0.2251 - accuracy: 0.9346 - val_loss: 0.2357 - val_accuracy: 0.9286 - 3s/epoch - 9ms/step
Epoch 9/20
336/336 - 3

In [ ]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

# We rely on ADAM as our optimizing methodology instead of Stochastic Gradient Descent (SGD)
adam = keras.optimizers.Adam(lr=learning_rate)
model2 = Model(Inp, output)

model2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
history2 = model2.fit(X_train, y_train,
                      batch_size = batch_size,
                      epochs = training_epochs,
                      verbose = 2,
                      validation_data=(X_cv, y_cv))

Epoch 1/20
336/336 - 5s - loss: 0.3398 - accuracy: 0.8975 - val_loss: 0.1877 - val_accuracy: 0.9442 - 5s/epoch - 15ms/step
Epoch 2/20
336/336 - 3s - loss: 0.1234 - accuracy: 0.9616 - val_loss: 0.1089 - val_accuracy: 0.9654 - 3s/epoch - 9ms/step
Epoch 3/20
336/336 - 3s - loss: 0.0781 - accuracy: 0.9758 - val_loss: 0.1073 - val_accuracy: 0.9658 - 3s/epoch - 9ms/step
Epoch 4/20
336/336 - 3s - loss: 0.0541 - accuracy: 0.9829 - val_loss: 0.1057 - val_accuracy: 0.9683 - 3s/epoch - 9ms/step
Epoch 5/20
336/336 - 4s - loss: 0.0450 - accuracy: 0.9863 - val_loss: 0.0924 - val_accuracy: 0.9755 - 4s/epoch - 12ms/step
Epoch 6/20
336/336 - 4s - loss: 0.0390 - accuracy: 0.9872 - val_loss: 0.0974 - val_accuracy: 0.9732 - 4s/epoch - 12ms/step
Epoch 7/20
336/336 - 4s - loss: 0.0307 - accuracy: 0.9896 - val_loss: 0.0991 - val_accuracy: 0.9731 - 4s/epoch - 13ms/step
Epoch 8/20
336/336 - 4s - loss: 0.0240 - accuracy: 0.9924 - val_loss: 0.1060 - val_accuracy: 0.9726 - 4s/epoch - 12ms/step
Epoch 9/20
336/336 

In [ ]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

learning_rate = 0.01  #made changes on the learning rate
adam = keras.optimizers.Adam(lr=learning_rate)
model2a = Model(Inp, output)

model2a.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history2a = model2a.fit(X_train, y_train,
                        batch_size = batch_size,
                        epochs = training_epochs,
                        verbose = 2,
                        validation_data=(X_cv, y_cv))

Epoch 1/20
336/336 - 4s - loss: 0.3320 - accuracy: 0.9007 - val_loss: 0.1699 - val_accuracy: 0.9494 - 4s/epoch - 13ms/step
Epoch 2/20
336/336 - 3s - loss: 0.1195 - accuracy: 0.9639 - val_loss: 0.1090 - val_accuracy: 0.9643 - 3s/epoch - 10ms/step
Epoch 3/20
336/336 - 3s - loss: 0.0802 - accuracy: 0.9744 - val_loss: 0.1094 - val_accuracy: 0.9668 - 3s/epoch - 9ms/step
Epoch 4/20
336/336 - 3s - loss: 0.0571 - accuracy: 0.9817 - val_loss: 0.1020 - val_accuracy: 0.9702 - 3s/epoch - 8ms/step
Epoch 5/20
336/336 - 3s - loss: 0.0450 - accuracy: 0.9850 - val_loss: 0.1021 - val_accuracy: 0.9710 - 3s/epoch - 10ms/step
Epoch 6/20
336/336 - 3s - loss: 0.0349 - accuracy: 0.9885 - val_loss: 0.1350 - val_accuracy: 0.9646 - 3s/epoch - 10ms/step
Epoch 7/20
336/336 - 3s - loss: 0.0284 - accuracy: 0.9907 - val_loss: 0.0892 - val_accuracy: 0.9758 - 3s/epoch - 9ms/step
Epoch 8/20
336/336 - 3s - loss: 0.0266 - accuracy: 0.9919 - val_loss: 0.0942 - val_accuracy: 0.9749 - 3s/epoch - 9ms/step
Epoch 9/20
336/336 -

In [ ]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

learning_rate = 0.5 #increase the learning rate
adam = keras.optimizers.Adam(lr=learning_rate)
model2b = Model(Inp, output)

model2b.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history2b = model2b.fit(X_train, y_train,
                        batch_size = batch_size,
                        epochs = training_epochs,
                            validation_data=(X_cv, y_cv))

Epoch 1/20
336/336 [==============================] - 7s 16ms/step - loss: 0.3375 - accuracy: 0.9012 - val_loss: 0.1455 - val_accuracy: 0.9573
Epoch 2/20
336/336 [==============================] - 3s 10ms/step - loss: 0.1290 - accuracy: 0.9601 - val_loss: 0.1266 - val_accuracy: 0.9593
Epoch 3/20
336/336 [==============================] - 3s 9ms/step - loss: 0.0802 - accuracy: 0.9749 - val_loss: 0.1177 - val_accuracy: 0.9644
Epoch 4/20
336/336 [==============================] - 3s 9ms/step - loss: 0.0578 - accuracy: 0.9818 - val_loss: 0.0956 - val_accuracy: 0.9711
Epoch 5/20
336/336 [==============================] - 4s 13ms/step - loss: 0.0456 - accuracy: 0.9858 - val_loss: 0.0991 - val_accuracy: 0.9718
Epoch 6/20
336/336 [==============================] - 3s 9ms/step - loss: 0.0328 - accuracy: 0.9892 - val_loss: 0.1043 - val_accuracy: 0.9721
Epoch 7/20
336/336 [==============================] - 3s 10ms/step - loss: 0.0279 - accuracy: 0.9915 - val_loss: 0.1053 - val_accuracy: 0.9732
Ep

In [ ]:
# Input Parameters - increase the hidden layer
n_input = 784 # number of features
n_hidden_1 = 300
n_hidden_2 = 100
n_hidden_3 = 100
n_hidden_4 = 100
n_hidden_5 = 200
num_digits = 10

In [ ]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
x = Dense(n_hidden_5, activation='relu', name = "Hidden_Layer_5")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

In [ ]:
# Our model would have '7' layers - input layer, 5 hidden layer and 1 output layer
model3 = Model(Inp, output)
model3.summary() # We have 308,010 parameters to estimate

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 784)]             0         
                                                                 
 Hidden_Layer_1 (Dense)      (None, 300)               235500    
                                                                 
 Hidden_Layer_2 (Dense)      (None, 100)               30100     
                                                                 
 Hidden_Layer_3 (Dense)      (None, 100)               10100     
                                                                 
 Hidden_Layer_4 (Dense)      (None, 100)               10100     
                                                                 
 Hidden_Layer_5 (Dense)      (None, 200)               20200     
                                                                 
 Output_Layer (Dense)        (None, 10)                2010

In [ ]:
# We rely on 'Adam' as our optimizing methodology
adam = keras.optimizers.Adam(lr=0.01)

model3.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history3 = model3.fit(X_train, y_train,
                      batch_size = batch_size,
                      epochs = training_epochs,
                      validation_data=(X_cv, y_cv))

Epoch 1/20
336/336 [==============================] - 6s 14ms/step - loss: 0.3688 - accuracy: 0.8861 - val_loss: 0.1548 - val_accuracy: 0.9539
Epoch 2/20
336/336 [==============================] - 3s 10ms/step - loss: 0.1284 - accuracy: 0.9611 - val_loss: 0.1442 - val_accuracy: 0.9571
Epoch 3/20
336/336 [==============================] - 3s 10ms/step - loss: 0.0878 - accuracy: 0.9727 - val_loss: 0.1152 - val_accuracy: 0.9639
Epoch 4/20
336/336 [==============================] - 3s 10ms/step - loss: 0.0638 - accuracy: 0.9795 - val_loss: 0.0992 - val_accuracy: 0.9685
Epoch 5/20
336/336 [==============================] - 4s 13ms/step - loss: 0.0512 - accuracy: 0.9839 - val_loss: 0.0978 - val_accuracy: 0.9738
Epoch 6/20
336/336 [==============================] - 3s 10ms/step - loss: 0.0438 - accuracy: 0.9855 - val_loss: 0.0954 - val_accuracy: 0.9727
Epoch 7/20
336/336 [==============================] - 3s 10ms/step - loss: 0.0324 - accuracy: 0.9902 - val_loss: 0.0944 - val_accuracy: 0.9758

In [ ]:
# Input Parameters
n_input = 784 # number of features
n_hidden_1 = 300
n_hidden_2 = 100
n_hidden_3 = 100
n_hidden_4 = 200
num_digits = 10

In [ ]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dropout(0.3)(x)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dropout(0.3)(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dropout(0.3)(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

In [ ]:
# Our model would have '6' layers - input layer, 4 hidden layer and 1 output layer
model4 = Model(Inp, output)
model4.summary() # We have 297,910 parameters to estimate

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 784)]             0         
                                                                 
 Hidden_Layer_1 (Dense)      (None, 300)               235500    
                                                                 
 dropout (Dropout)           (None, 300)               0         
                                                                 
 Hidden_Layer_2 (Dense)      (None, 100)               30100     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 Hidden_Layer_3 (Dense)      (None, 100)               10100     
                                                                 
 dropout_2 (Dropout)         (None, 100)               0   

In [ ]:
model4.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history = model4.fit(X_train, y_train,
                    batch_size = batch_size,
                    epochs = training_epochs,
                    validation_data=(X_cv, y_cv))

Epoch 1/20
336/336 [==============================] - 7s 13ms/step - loss: 0.5928 - accuracy: 0.8096 - val_loss: 0.1808 - val_accuracy: 0.9450
Epoch 2/20
336/336 [==============================] - 4s 12ms/step - loss: 0.2317 - accuracy: 0.9319 - val_loss: 0.1472 - val_accuracy: 0.9557
Epoch 3/20
336/336 [==============================] - 4s 10ms/step - loss: 0.1782 - accuracy: 0.9475 - val_loss: 0.1197 - val_accuracy: 0.9651
Epoch 4/20
336/336 [==============================] - 3s 10ms/step - loss: 0.1438 - accuracy: 0.9588 - val_loss: 0.1162 - val_accuracy: 0.9670
Epoch 5/20
336/336 [==============================] - 5s 14ms/step - loss: 0.1243 - accuracy: 0.9629 - val_loss: 0.1025 - val_accuracy: 0.9704
Epoch 6/20
336/336 [==============================] - 4s 11ms/step - loss: 0.1086 - accuracy: 0.9680 - val_loss: 0.0902 - val_accuracy: 0.9754
Epoch 7/20
336/336 [==============================] - 3s 10ms/step - loss: 0.0969 - accuracy: 0.9719 - val_loss: 0.0955 - val_accuracy: 0.9758

In [ ]:
test_pred = pd.DataFrame(model4.predict(X_test, batch_size=200)) #takes the test data as input and returns an array of predicted probabilities for each class (digit) for every test image.
test_pred = pd.DataFrame(test_pred.idxmax(axis = 1))  #predicted labels (class indices) for the test data.
test_pred.index.name = 'ImageId'  #'ImageId' represents the image number (starting from 1) and 'Label' represents the predicted digit (class) for each test image.
test_pred = test_pred.rename(columns = {0: 'Label'}).reset_index()
test_pred['ImageId'] = test_pred['ImageId'] + 1

test_pred.head()

140/140 [==============================] - 1s 5ms/step


,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3


In [ ]:
test_pred.to_csv('mnist_submission.csv', index = False)

In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv('mnist_submission.csv')

In [ ]:
df

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3
...,...,...
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9


import numpy as np
from google.colab import autoviz
df_8678906292410673922 = autoviz.get_registered_df('df_8678906292410673922')

def value_plot(df, y, sort_ascending=False, figsize=(2, 1)):
  from matplotlib import pyplot as plt
  if sort_ascending:
    df = df.sort_values(y).reset_index(drop=True)
  _, ax = plt.subplots(figsize=figsize)
  df[y].plot(kind='line')
  plt.title(y)
  ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(df_8678906292410673922, *['ImageId'], **{})
chart

import numpy as np
from google.colab import autoviz
df_8678906292410673922 = autoviz.get_registered_df('df_8678906292410673922')

def value_plot(df, y, sort_ascending=False, figsize=(2, 1)):
  from matplotlib import pyplot as plt
  if sort_ascending:
    df = df.sort_values(y).reset_index(drop=True)
  _, ax = plt.subplots(figsize=figsize)
  df[y].plot(kind='line')
  plt.title(y)
  ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(df_8678906292410673922, *['Label'], **{})
chart

import numpy as np
from google.colab import autoviz
df_8678906292410673922 = autoviz.get_registered_df('df_8678906292410673922')

def histogram(df, colname, num_bins=20, figsize=(2, 1)):
  from matplotlib import pyplot as plt
  _, ax = plt.subplots(figsize=figsize)
  plt.hist(df[colname], bins=num_bins, histtype='stepfilled')
  plt.ylabel('count')
  plt.title(colname)
  ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(df_8678906292410673922, *['ImageId'], **{})
chart

import numpy as np
from google.colab import autoviz
df_8678906292410673922 = autoviz.get_registered_df('df_8678906292410673922')

def histogram(df, colname, num_bins=20, figsize=(2, 1)):
  from matplotlib import pyplot as plt
  _, ax = plt.subplots(figsize=figsize)
  plt.hist(df[colname], bins=num_bins, histtype='stepfilled')
  plt.ylabel('count')
  plt.title(colname)
  ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(df_8678906292410673922, *['Label'], **{})
chart

import numpy as np
from google.colab import autoviz
df_8678906292410673922 = autoviz.get_registered_df('df_8678906292410673922')

def scatter_plots(df, colname_pairs, scatter_plot_size=2.5, size=8, alpha=.6):
  from matplotlib import pyplot as plt
  plt.figure(figsize=(len(colname_pairs) * scatter_plot_size, scatter_plot_size))
  for plot_i, (x_colname, y_colname) in enumerate(colname_pairs, start=1):
    ax = plt.subplot(1, len(colname_pairs), plot_i)
    ax.scatter(df[x_colname], df[y_colname], s=size, alpha=alpha)
    plt.xlabel(x_colname)
    plt.ylabel(y_colname)
    ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = scatter_plots(df_8678906292410673922, *[[['ImageId', 'Label']]], **{})
chart

import numpy as np
from google.colab import autoviz
df_8678906292410673922 = autoviz.get_registered_df('df_8678906292410673922')

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figsize=(2.5, 1.3), mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(df_8678906292410673922, *['ImageId', 'Label', None], **{})
chart

import numpy as np
from google.colab import autoviz
df_8678906292410673922 = autoviz.get_registered_df('df_8678906292410673922')

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figsize=(2.5, 1.3), mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(df_8678906292410673922, *['ImageId', 'count()', None], **{})
chart